In [1]:
from typing import Tuple

import numpy as np
import pandas as pd
from IPython.core.display import Markdown
from IPython.core.display_functions import display

import notebooks.utils as nb_utils
import src.config as config
from src.config import FeaturesInfo
from src.db.broker import DbBroker
from src.features.univariate_analysis import UACleaner
from src.features.utils import downcast_nonnumerical_dtypes
from src.logger import logging
from src.utils import initialize_features_info, preprocess_init

In [2]:
%load_ext autoreload
%autoreload 2

## Get dataset and metadata

In [3]:
def get_dataset_from_db() -> pd.DataFrame:
    db_broker = DbBroker()
    df = pd.read_sql(
        db_broker.get_all_listings_statement(),
        db_broker.engine,
        dtype_backend=config.DTYPE_BACKEND,
        index_col=config.INDEX,
    )
    df = df.rename(str, axis="columns")
    db_broker.engine.dispose()
    return df


def get_metadata() -> FeaturesInfo:
    features_info = initialize_features_info()
    return features_info

In [4]:
df = get_dataset_from_db()
features_info = get_metadata()

## Dataset info

In [5]:
print("Entire dataset")
print(f"Duplicate rows: {df.duplicated().sum()}")
print(20 * "-")
df.info()

Entire dataset
Duplicate rows: 0
--------------------
<class 'pandas.core.frame.DataFrame'>
Index: 30788 entries, 9249043 to 23246511
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   name                      30788 non-null  string
 1   short_url                 30788 non-null  string
 2   price                     30788 non-null  string
 3   listing_followers_no      30788 non-null  string
 4   location                  30788 non-null  string
 5   images_no                 30788 non-null  string
 6   safety                    30788 non-null  string
 7   equipment                 30788 non-null  string
 8   other                     30788 non-null  string
 9   description               30788 non-null  string
 10  id_1                      30788 non-null  Int64 
 11  condition                 30788 non-null  string
 12  brand                     30788 non-null  string
 13  model             

## Entire dataset cleaning

### Initial clean

In [6]:
@preprocess_init
def initial_clean_nb(
    df: pd.DataFrame, features_info: FeaturesInfo
) -> Tuple[pd.DataFrame, FeaturesInfo]:
    df.columns = df.columns.astype("string")
    print("Transformed column type to string")

    # Prefix columns from table general_informations with "gi_"
    # and additional_informations with "ai"
    id_1_col_idx = df.columns.get_loc("id_1")
    id_2_col_idx = df.columns.get_loc("id_2")

    columns_from_gi = df.columns[id_1_col_idx + 1 : id_2_col_idx].values
    columns_from_ai = df.columns[id_2_col_idx + 1 :].values
        
    df.rename(
        columns=dict(zip(columns_from_gi, "gi_" + columns_from_gi)), inplace=True
    )
    print("Added prefix 'gi_' to columns from table general_informations")
    df.rename(
        columns=dict(zip(columns_from_ai, "ai_" + columns_from_ai)), inplace=True
    )
    print("Added prefix 'ai_' to columns from table additional_informations")

    del df["id_1"], df["id_2"]
    print("Removed redundant ids")

    del df["gi_fixed_price"]
    print("Removing gi_fixed_price that is poorly scraped")

    return df, features_info

In [7]:
df, features_info = initial_clean_nb(df=df, features_info=features_info)

Transformed column type to string
Added prefix 'gi_' to columns from table general_informations
Added prefix 'ai_' to columns from table additional_informations
Removed redundant ids
Removing gi_fixed_price that is poorly scraped


### Feature 'name'

In [8]:
@preprocess_init
def cf_name_nb(
    df: pd.DataFrame, features_info: FeaturesInfo
) -> Tuple[pd.DataFrame, FeaturesInfo]:
    feature_name = nb_utils.get_feature_name()

    print("First 5 rows")
    display(df[[feature_name]].head())
    print()

    print("Counts by frequencies for the first 20 categories")
    display(df[feature_name].value_counts().head(20))
    print()

    sum_irregular_name_rows = (
        df[feature_name].str.contains("ucesc|učešć", case=False).sum()
    )
    print(f"Irregular 'name' rows count: {sum_irregular_name_rows}")
    display(Markdown("This feature will be left for the Multivariate analysis."))
    print()

    features_info["other"].append(feature_name)
    print(f"Added {feature_name} to 'other' features")
    
    return df, features_info

In [9]:
df, features_info = cf_name_nb(df=df, features_info=features_info)

First 5 rows


,name
id,
9249043,Volvo 940 DIPLOMATA NOV
9650557,"SsangYong Kyron 2,0 XDI4x4RESTAJLING"
9657017,Audi A4 2.5 TDI Quattro CH
9755964,Dacia Sandero 1.0 TCE 90 LAGER
9921885,Zastava 750 presla 38.400



Counts by frequencies for the first 20 categories


name
Audi A4                 179
Volkswagen Golf 5       123
Audi A6                 119
BMW 320                 115
Volkswagen Polo         114
Volkswagen Passat B6    106
Audi A3                 105
Renault Megane          102
Škoda Octavia           100
Volkswagen Golf 6        98
Volkswagen Golf 7        87
Ford Focus               82
Renault Clio             79
BMW 520                  73
Ford Fiesta              69
Fiat Punto               68
Opel Corsa D             68
Peugeot 308              67
Citroen C5               65
Audi A4 2.0 TDI          64
Name: count, dtype: Int64


Irregular 'name' rows count: 18


This feature will be left for the Multivariate analysis.


Added name to 'other' features


### Feature 'short_url'

In [10]:
@preprocess_init
def cf_short_url_nb(
    df: pd.DataFrame, features_info: FeaturesInfo
) -> Tuple[pd.DataFrame, FeaturesInfo]:
    feature_name = nb_utils.get_feature_name()

    print("Display first 5 rows")
    display(df[[feature_name]].head())
    print()

    features_info["other"].append(feature_name)
    print(f"Added {feature_name} to 'other' features")
    
    return df, features_info

In [11]:
df, features_info = cf_short_url_nb(df=df, features_info=features_info)

Display first 5 rows


,short_url
id,
9249043,https://www.polovniautomobili.com/auto-oglasi/...
9650557,https://www.polovniautomobili.com/auto-oglasi/...
9657017,https://www.polovniautomobili.com/auto-oglasi/...
9755964,https://www.polovniautomobili.com/auto-oglasi/...
9921885,https://www.polovniautomobili.com/auto-oglasi/...



Added short_url to 'other' features


### Feature 'price'

In [12]:
@preprocess_init
def cf_price_nb(
    df: pd.DataFrame, features_info: FeaturesInfo
) -> Tuple[pd.DataFrame, FeaturesInfo]:
    feature_name = nb_utils.get_feature_name()

    print("First 5 rows")
    display(df[[feature_name]].head())
    print()

    df[feature_name] = pd.to_numeric(
        df[feature_name].str.slice(stop=-1).str.replace(".", ""),
        errors="raise",
        downcast="unsigned",
    )
    print("Removed '.' from values and transformed to numerical")
    print()

    print("Cars where price = 1")
    display(df[df[feature_name] == 1])
    display(
        Markdown(
            "These are cars where price='Po dogovoru' on the website, so that means they didn't have a price to begin with and they will be dropped."
        )
    )
    print()

    df = df[df[feature_name] != 1]
    print("Removed cars that had price = 1")

    print("Cars where price = 'Po dogovoru'")
    display(df[df[feature_name] == "Po dogovoru"])
    print()

    cars_price_less_than_100 = df.loc[df[feature_name] < 100, feature_name]
    print("Short url of cars that had price < 100")
    display(df.loc[cars_price_less_than_100.index, "short_url"].tolist())
    print()

    display(
        Markdown(
            "All of these cars are going to be deleted because they actually don't have a value of price (their current value is the result of poor handling the case when price is set by an agreement)."
        )
    )

    df = df.drop(cars_price_less_than_100.index, axis=0)
    print("Removed cars that had price < 100")

    return df, features_info

In [13]:
df, features_info = cf_price_nb(df=df, features_info=features_info)

First 5 rows


,price
id,
9249043,18.000 €
9650557,4.500 €
9657017,3.300 €
9755964,16.100 €
9921885,12.999 €



Removed '.' from values and transformed to numerical

Cars where price = 1


,name,short_url,price,listing_followers_no,location,images_no,safety,equipment,other,description,...,ai_import_country,ai_sales_method,ai_credit,ai_deposit,ai_installment_no,ai_installment_amount,ai_interest_free_credit,ai_leasing,ai_cash_payment,ai_range_on_full_battery_km
id,,,,,,,,,,,,,,,,,,,,,
23220124,Audi A4 RATA 183e BEZ UCESCA,https://www.polovniautomobili.com/auto-oglasi/...,1,2,Beograd,39,"Airbag za vozača,Airbag za suvozača,Bočni airb...","Servo volan,Daljinsko zaključavanje,Tonirana s...",,,...,<NA>,<NA>,DA,<NA>,<NA>,183 €,<NA>,<NA>,<NA>,<NA>
23220566,Renault Scenic RATA 148e BEZ UCESCA,https://www.polovniautomobili.com/auto-oglasi/...,1,,Beograd,28,"Airbag za vozača,Airbag za suvozača,Bočni airb...","Servo volan,Multifunkcionalni volan,Tempomat,D...",,VOZILO U DOLASKU ...............................,...,<NA>,<NA>,DA,<NA>,<NA>,148 €,<NA>,<NA>,<NA>,<NA>
23221119,Opel Astra J RATA 148e BEZ UCESCA,https://www.polovniautomobili.com/auto-oglasi/...,1,1,Beograd,14,"Airbag za vozača,Airbag za suvozača,Bočni airb...","Servo volan,Multifunkcionalni volan,Tempomat,D...",,Vozilo u dolasku ...............................,...,<NA>,<NA>,DA,<NA>,<NA>,148 €,<NA>,<NA>,<NA>,<NA>
23228430,Toyota Yaris RATA 78e BEZ UCESCA,https://www.polovniautomobili.com/auto-oglasi/...,1,1,Beograd,17,"Airbag za vozača,Airbag za suvozača,Bočni airb...","Servo volan,Daljinsko zaključavanje,Putni raču...",,VOZILO U DOLASKU,...,<NA>,<NA>,DA,<NA>,<NA>,78 €,<NA>,<NA>,<NA>,<NA>
23230448,Peugeot 306,https://www.polovniautomobili.com/auto-oglasi/...,1,2,Landol,19,"Airbag za vozača,Airbag za suvozača,ABS","Metalik boja,Servo volan,Tonirana stakla,Radio...",Garažiran,,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


These are cars where price='Po dogovoru' on the website, so that means they didn't have a price to begin with and they will be dropped.

...

,name,short_url,price,listing_followers_no,location,images_no,safety,equipment,other,description,...,ai_import_country,ai_sales_method,ai_credit,ai_deposit,ai_installment_no,ai_installment_amount,ai_interest_free_credit,ai_leasing,ai_cash_payment,ai_range_on_full_battery_km
id,,,,,,,,,,,,,,,,,,,,,



Short url of cars that had price < 100


['https://www.polovniautomobili.com/auto-oglasi/22675403/Opel Meriva RATA 78e BEZ UCESCA',
 'https://www.polovniautomobili.com/auto-oglasi/22681873/Ford Focus RATA 83e BEZ UČEŠĆA',
 'https://www.polovniautomobili.com/auto-oglasi/22681890/Opel Astra H RATA 99e BEZ UČEŠĆA',
 'https://www.polovniautomobili.com/auto-oglasi/23225453/Zastava Yugo Tempo']

All of these cars are going to be deleted because they actually don't have a value of price (their current value is the result of poor handling the case when price is set by an agreement).

Removed cars that had price < 100


### Feature 'listing_followers_no'

In [14]:
@preprocess_init
def cf_listing_followers_no_nb(
    df: pd.DataFrame, features_info: FeaturesInfo
) -> Tuple[pd.DataFrame, FeaturesInfo]:
    feature_name = nb_utils.get_feature_name()

    print("First 5 rows")
    display(df[[feature_name]].head())
    print()

    if (df[feature_name] == "").sum() == pd.to_numeric(
        df[feature_name], downcast="unsigned"
    ).isna().sum():
        df[feature_name] = pd.to_numeric(df[feature_name], downcast="unsigned")
    else:
        raise ValueError(
            "There is a listing_followers_no value that is probably incorrectly parsed."
        )
    print("Transformed to numerical")

    features_info["numerical"].append(feature_name)
    print(f"Added '{feature_name}' to numerical features'")

    return df, features_info

In [15]:
df, features_info = cf_listing_followers_no_nb(df=df, features_info=features_info)

First 5 rows


,listing_followers_no
id,
9249043,58
9650557,87
9657017,151
9755964,50
9921885,421



Transformed to numerical
Added 'listing_followers_no' to numerical features'


### Feature 'location'

In [16]:
@preprocess_init
def cf_location_nb(
    df: pd.DataFrame, features_info: FeaturesInfo
) -> Tuple[pd.DataFrame, FeaturesInfo]:
    feature_name = nb_utils.get_feature_name()

    print("First 5 rows")
    display(df[[feature_name]].head())
    print()

    df[feature_name] = pd.Categorical(df[feature_name], ordered=False)
    print(f"Converted '{feature_name}' to categorical type (nominal)")

    features_info["nominal"].append(feature_name)
    print(f"Added '{feature_name}' to 'nominal' features")
    
    return df, features_info

In [17]:
df, features_info = cf_location_nb(df=df, features_info=features_info)

First 5 rows


,location
id,
9249043,Vrbas
9650557,Novi Sad
9657017,Loznica
9755964,Novi Sad
9921885,Zemun



Converted 'location' to categorical type (nominal)
Added 'location' to 'nominal' features


### Feature 'images_no'

In [18]:
@preprocess_init
def cf_images_no_nb(
    df: pd.DataFrame, features_info: FeaturesInfo
) -> Tuple[pd.DataFrame, FeaturesInfo]:
    feature_name = nb_utils.get_feature_name()

    print("First 5 rows")
    display(df[[feature_name]].head())
    print()

    na_count = pd.to_numeric(df[feature_name], downcast="unsigned").isna().sum()
    if na_count > 0:
        raise ValueError(f"Check why {feature_name} contains {na_count} values!")
    print("Checked if casting would be is correct")

    df[feature_name] = pd.to_numeric(df[feature_name], downcast="unsigned")
    print("Transformed to numerical")

    features_info["numerical"].append(feature_name)
    print(f"Added {feature_name} to 'numerical' features")

    return df, features_info

In [19]:
df, features_info = cf_images_no_nb(df=df, features_info=features_info)

First 5 rows


,images_no
id,
9249043,30
9650557,17
9657017,19
9755964,9
9921885,15



Checked if casting would be is correct
Transformed to numerical
Added images_no to 'numerical' features


### Feature 'safety'

In [20]:
@preprocess_init
def cf_safety_nb(
    df: pd.DataFrame, features_info: FeaturesInfo
) -> Tuple[pd.DataFrame, FeaturesInfo]:
    feature_name = nb_utils.get_feature_name()

    print("First 5 rows")
    display(df[[feature_name]].head())
    print()

    prefix = "s_"

    # Create data frame with dummy columns
    df_safety_dummies = df[feature_name].str.get_dummies(sep=",").add_prefix(prefix)
    df = pd.concat([df, df_safety_dummies], axis=1)
    print("Extended the data frame with dummy columns")

    del df[feature_name]
    print(f"Deleted {feature_name} column") 

    safety_columns = [col for col in df.columns if col.startswith(prefix)]
    df[safety_columns] = df[safety_columns].astype("boolean")
    print("Converted all remaining safety columns to boolean")

    features_info["binary"].extend(safety_columns)
    print("Added all remaining safety columns to 'binary' features")

    return df, features_info

In [21]:
df, features_info = cf_safety_nb(df=df, features_info=features_info)

First 5 rows


,safety
id,
9249043,
9650557,"Airbag za vozača,Airbag za suvozača,Bočni airb..."
9657017,"Airbag za vozača,Airbag za suvozača,Bočni airb..."
9755964,"Airbag za vozača,Airbag za suvozača,Bočni airb..."
9921885,



Extended the data frame with dummy columns
Deleted safety column
Converted all remaining safety columns to boolean
Added all remaining safety columns to 'binary' features


### Feature 'equipment'

In [22]:
@preprocess_init
def cf_equipment_nb(
    df: pd.DataFrame, features_info: FeaturesInfo
) -> Tuple[pd.DataFrame, FeaturesInfo]:
    feature_name = nb_utils.get_feature_name()

    print("First 5 rows")
    display(df[[feature_name]].head())
    print()

    prefix = "e_"

    # Create data frame with dummy columns
    df_equipment_dummies = df[feature_name].str.get_dummies(sep=",").add_prefix(prefix)
    df = pd.concat([df, df_equipment_dummies], axis=1)
    print("Extended the data frame with dummy columns")

    del df[feature_name]
    print(f"Deleted {feature_name} column") 

    equipment_columns = df_equipment_dummies.columns.tolist()

    df[equipment_columns] = df[equipment_columns].astype("boolean")
    print("Converted all remaining equipment columns to boolean")

    features_info["binary"].extend(equipment_columns)
    print("Added all remaining equipment columns to 'binary' features")

    return df, features_info


In [23]:
df, features_info = cf_equipment_nb(df=df, features_info=features_info)

First 5 rows


,equipment
id,
9249043,
9650557,"Metalik boja,Branici u boji auta,Servo volan,M..."
9657017,"Metalik boja,Branici u boji auta,Servo volan,T..."
9755964,"Metalik boja,Branici u boji auta,Servo volan,M..."
9921885,Svetla za maglu



Extended the data frame with dummy columns
Deleted equipment column
Converted all remaining equipment columns to boolean
Added all remaining equipment columns to 'binary' features


### Feature 'other'

In [24]:
@preprocess_init
def cf_other_nb(
    df: pd.DataFrame, features_info: FeaturesInfo
) -> Tuple[pd.DataFrame, FeaturesInfo]:
    feature_name = nb_utils.get_feature_name()

    print("First 5 rows")
    display(df[[feature_name]].head())
    print()

    prefix = "o_"

    # Create data frame with dummy columns
    df_other_dummies = df[feature_name].str.get_dummies(sep=",").add_prefix(prefix)
    df = pd.concat([df, df_other_dummies], axis=1)
    print("Extended the data frame with dummy columns")

    del df[feature_name]
    print(f"Deleted {feature_name} column") 

    other_columns = df_other_dummies.columns.tolist()
    
    df[other_columns] = df[other_columns].astype("boolean")
    print("Converted all remaining other columns to boolean")
    
    features_info["binary"].extend(other_columns)
    print("Added all other columns to 'binary' features")

    return df, features_info


In [25]:
df, features_info = cf_other_nb(df=df, features_info=features_info)

First 5 rows


,other
id,
9249043,
9650557,Rezervni ključ
9657017,"Servisna knjiga,Rezervni ključ"
9755964,"Prvi vlasnik,Kupljen nov u Srbiji,Garancija,Ga..."
9921885,"Kupljen nov u Srbiji,Garancija,Garažiran,Servi..."



Extended the data frame with dummy columns
Deleted other column
Converted all remaining other columns to boolean
Added all other columns to 'binary' features


### Feature 'description'

In [26]:
@preprocess_init
def cf_description_nb(
    df: pd.DataFrame, features_info: FeaturesInfo
) -> Tuple[pd.DataFrame, FeaturesInfo]:
    feature_name = nb_utils.get_feature_name()

    print("First 5 rows")
    display(df[[feature_name]].head())
    print()

    features_info["other"].append(feature_name)
    print(f"Added '{feature_name}' to 'other' features")

    return df, features_info

In [27]:
df, features_info = cf_description_nb(df=df, features_info=features_info)

First 5 rows


,description
id,
9249043,"DIPLOMATA,JEDINSTVEN SA MALOM KILOMETRAZOM,(25..."
9650557,Vozilo u odlicnom stanju ...
9657017,- A4 Quattro - manuelni menjac - Uvoz iz Svjca...
9755964,"NOVO VOZILO, ODLICNI USLOVI FINANSIRANJA . OVL..."
9921885,Fica ima pravih 38.400 km !!! Nikad nije nist...



Added 'description' to 'other' features


### "GeneralInformation" features

In [28]:
@preprocess_init
def c_general_informations_nb(
    df: pd.DataFrame, features_info: FeaturesInfo
) -> Tuple[pd.DataFrame, FeaturesInfo]:
    pd.set_option("mode.chained_assignment", None)

    gi_columns = [
        "gi_condition",
        "gi_brand",
        "gi_model",
        "gi_production_year",
        "gi_kilometerage",
        "gi_body_type",
        "gi_fuel_type",
        "gi_engine_capacity",
        "gi_engine_power",
        "gi_trade_in",
        "gi_certified",
        "gi_battery_capacity",
    ]

    print("Frequencies for GeneralInformation columns")
    for col in gi_columns:
        display(nb_utils.get_value_counts_freq_with_perc(df, col))

    display(
        Markdown(
            "Only used cars will be left in the dataset, so all new cars are going to be deleted."
        )
    )

    new_cars_cond = df.gi_condition == "Novo vozilo"
    df = df.loc[~new_cars_cond, :]
    print(f"Deleted from df {new_cars_cond.sum()} rows that represent new cars")

    del df["gi_condition"]
    print("Deleted 'gi_condition' feature")

    df.gi_kilometerage = pd.to_numeric(
        df.gi_kilometerage.str.rstrip("km").str.replace(".", ""), downcast="unsigned"
    )
    print("Stripped 'km', removed '.' and converted 'gi_kilometerage' to numerical")

    df.gi_production_year = pd.to_numeric(
        df.gi_production_year.str.rstrip("."), downcast="unsigned"
    )
    print("Removed '.' and converted 'gi_production_year' to numerical")

    df.gi_engine_capacity = pd.to_numeric(
        df.gi_engine_capacity.str.rstrip("cm3"), errors="raise", downcast="unsigned"
    )
    print("Stripped 'cm3' and converted 'gi_engine_capacity' to numerical")

    df.gi_engine_power = pd.to_numeric(
        df.gi_engine_power.str.split("/", n=1).str.get(0), downcast="unsigned"
    )
    print("Extracted only value of kW (ignore KS which stands for horse powers)")

    df.gi_battery_capacity = pd.to_numeric(
        df.gi_battery_capacity.str.rstrip("kWh"), downcast="unsigned"
    )
    print("Stripped 'kWh' and converted 'gi_battery_capacity' to numerical")

    nominal_cols = [
        "gi_brand",
        "gi_model",
        "gi_body_type",
        "gi_fuel_type",
        "gi_trade_in",
    ]
    numerical_cols = [
        "gi_kilometerage",
        "gi_production_year",
        "gi_engine_capacity",
        "gi_engine_power",
        "gi_battery_capacity",
    ]
    other_cols = ["gi_certified"]

    for col in nominal_cols:
        df[col] = pd.Categorical(df[col], ordered=False)
    print("Converted nominal columns to categorical type (nominal)")

    features_info["nominal"].extend(nominal_cols)
    print(f"Added {nominal_cols} columns to 'nominal' features")

    features_info["numerical"].extend(numerical_cols)
    print(f"Added {numerical_cols} columns to 'numerical' features")

    features_info["other"].extend(other_cols)
    print(f"Added {other_cols} columns to 'other' features")

    pd.set_option("mode.chained_assignment", "warn")

    return df, features_info

In [29]:
df, features_info = c_general_informations_nb(df=df, features_info=features_info)

Frequencies for GeneralInformation columns


,count,percentage [%]
gi_condition,,
Polovno vozilo,30135,97.907664
Novo vozilo,644,2.092336


,count,percentage [%]
gi_brand,,
Volkswagen,4811,15.630787
Audi,3189,10.36096
BMW,2770,8.999643
Opel,2377,7.722798
Peugeot,2055,6.67663
...,...,...
Oldsmobile,1,0.003249
Jinpeng,1,0.003249
Lamborghini,1,0.003249


,count,percentage [%]
gi_model,,
A4,866,2.813607
Octavia,630,2.04685
A3,628,2.040352
A6,621,2.017609
Golf 7,574,1.864908
...,...,...
ML 420,1,0.003249
469,1,0.003249
bZ4X,1,0.003249


,count,percentage [%]
gi_production_year,,
2008.,1984,6.445953
2011.,1976,6.419962
2010.,1871,6.07882
2009.,1838,5.971604
2007.,1833,5.955359
...,...,...
1957.,1,0.003249
1961.,1,0.003249
1973.,1,0.003249


...

,count,percentage [%]
gi_trade_in,,
NE,22580,73.361708
DA,3740,12.151142
DA (za jeftinije),2827,9.184834
DA (za skuplje),991,3.219728
DA (u istoj ceni),438,1.423048
DA (staro za novo),203,0.659541


,count,percentage [%]
gi_certified,,
<NA>,29087,94.502745
Nije atestiran,164,0.532831
do: 10.2027,54,0.175444
do: 07.2027,49,0.159199
do: 01.2028,49,0.159199
...,...,...
do: 10.2032,1,0.003249
do: 10.2030,1,0.003249
do: 04.2031,1,0.003249


,count,percentage [%]
gi_battery_capacity,,
<NA>,30598,99.411937
17 kWh,11,0.035739
75 kWh,10,0.03249
58 kWh,7,0.022743
22 kWh,7,0.022743
...,...,...
21.6 kWh,1,0.003249
38.3 kWh,1,0.003249
72.6 kWh,1,0.003249


Only used cars will be left in the dataset, so all new cars are going to be deleted.

Deleted from df 644 rows that represent new cars
Deleted 'gi_condition' feature
Stripped 'km', removed '.' and converted 'gi_kilometerage' to numerical
Removed '.' and converted 'gi_production_year' to numerical
Stripped 'cm3' and converted 'gi_engine_capacity' to numerical
Extracted only value of kW (ignore KS which stands for horse powers)
Stripped 'kWh' and converted 'gi_battery_capacity' to numerical
Converted nominal columns to categorical type (nominal)
Added ['gi_brand', 'gi_model', 'gi_body_type', 'gi_fuel_type', 'gi_trade_in'] columns to 'nominal' features
Added ['gi_kilometerage', 'gi_production_year', 'gi_engine_capacity', 'gi_engine_power', 'gi_battery_capacity'] columns to 'numerical' features
Added ['gi_certified'] columns to 'other' features


In [30]:
df.info(max_cols=200, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 30135 entries, 9249043 to 23246511
Data columns (total 171 columns):
 #    Column                                   Non-Null Count  Dtype   
---   ------                                   --------------  -----   
 0    name                                     30135 non-null  string  
 1    short_url                                30135 non-null  string  
 2    price                                    30135 non-null  UInt32  
 3    listing_followers_no                     27334 non-null  UInt16  
 4    location                                 30135 non-null  category
 5    images_no                                30135 non-null  UInt8   
 6    description                              30135 non-null  string  
 7    gi_brand                                 30135 non-null  category
 8    gi_model                                 30135 non-null  category
 9    gi_production_year                       30135 non-null  UInt16  
 10   gi_kilometerage 

### "AdditionalInformation" features

In [ ]:
ai_columns = [col for col in df_train.columns if "ai_" in col]

print("Descriptives for AdditionalInformation columns")
display(df_train[ai_columns].describe())
print()
print("Value counts for AdditionalInformation columns")
for col in ai_columns:
    display(nb_utils.get_value_counts_freq_with_perc(df_train, col))
del ai_columns

Descriptives for AdditionalInformation columns


,ai_floating_flywheel,ai_engine_emission_class,ai_propulsion,ai_gearbox_type,ai_doors_no,ai_seats_no,ai_steering_wheel_side,ai_air_conditioning,ai_color,ai_interior_material,...,ai_import_country,ai_sales_method,ai_credit,ai_deposit,ai_installment_no,ai_installment_amount,ai_interest_free_credit,ai_leasing,ai_cash_payment,ai_range_on_full_battery_km
count,7689,24096,24102,24101,24101,24102,24102,24102,24102,20808,...,6089,2451,1457,388,491,468,151,626,231,97
unique,2,6,4,6,2,8,2,3,19,5,...,23,2,1,181,21,242,1,1,138,49
top,Sa plivajućim zamajcem,Euro 4,Prednji,Manuelni 5 brzina,4/5 vrata,5 sedišta,Levi volan,Automatska klima,Siva,Štof,...,Švajcarska,Komisiona,DA,30 €,84,115 €,DA,DA,5000 €,300
freq,4957,7188,17490,8491,21613,21599,23991,16345,7749,12997,...,1632,2228,1457,26,271,17,151,626,9,9



Value counts for AdditionalInformation columns


,count,percentage [%]
ai_floating_flywheel,,
<NA>,16413,68.098083
Sa plivajućim zamajcem,4957,20.566758
Bez plivajućeg zamajca,2732,11.335159


,count,percentage [%]
ai_engine_emission_class,,
Euro 4,7188,29.823251
Euro 6,7148,29.65729
Euro 5,6685,27.736287
Euro 3,2549,10.575886
Euro 2,312,1.294498
Euro 1,214,0.887893
<NA>,6,0.024894


...

,count,percentage [%]
ai_installment_amount,,
<NA>,23634,98.058252
115 €,17,0.070534
140 €,9,0.037341
150 €,9,0.037341
94 €,8,0.033192
...,...,...
183 €,1,0.004149
96 €,1,0.004149
236 €,1,0.004149


,count,percentage [%]
ai_interest_free_credit,,
<NA>,23951,99.373496
DA,151,0.626504


,count,percentage [%]
ai_leasing,,
<NA>,23476,97.402705
DA,626,2.597295


,count,percentage [%]
ai_cash_payment,,
<NA>,23871,99.041573
5000 €,9,0.037341
3000 €,7,0.029043
6000 €,7,0.029043
4800 €,6,0.024894
...,...,...
1245 €,1,0.004149
14000 €,1,0.004149
6700 €,1,0.004149


,count,percentage [%]
ai_range_on_full_battery_km,,
<NA>,24005,99.597544
300,9,0.037341
140,6,0.024894
160,5,0.020745
150,5,0.020745
500,4,0.016596
120,4,0.016596
450,3,0.012447
130,3,0.012447


In [ ]:
df_train.ai_engine_emission_class = df_train.ai_engine_emission_class.str.strip("Euro")

df_train.replace(
    {
        "ai_gearbox_type": {
            "Automatski": "Automatski / poluautomatski",
            "Poluautomatski": "Automatski / poluautomatski",
        }
    },
    inplace=True,
)

# df_train.fillna({"ai_doors_no": df_train.ai_doors_no.mode()[0]}, inplace=True)

df_train.ai_doors_no = pd.to_numeric(df_train.ai_doors_no.replace({"4/5 vrata": "1", "2/3 vrata": "0"}))

df_train.ai_seats_no = pd.to_numeric(df_train.ai_seats_no.str.strip("sedišta"))

df_train = df_train.loc[df_train.ai_steering_wheel_side != "Desni volan", :]
del df_train["ai_steering_wheel_side"]

df_train.replace(
    {
        "ai_damage": {
            "Nije oštećen": "0",
            "Oštećen - u voznom stanju": "1",
            "Oštećen - nije u voznom stanju": "2",
        }
    },
    inplace=True,
)

features_info["binary"].extend(["ai_doors_no"])
features_info["ordinal"].extend(["ai_engine_emission_class", "ai_damage"])
features_info["nominal"].extend(
    [
        "ai_floating_flywheel",
        "ai_gearbox_type",
        "ai_air_conditioning",
        "ai_color",
        "ai_interior_material",
        "ai_interior_color",
        "ai_propulsion",
        "ai_vehicle_origin",
        "ai_ownership",
    ]
)
features_info["numerical"].extend(["ai_seats_no"])
features_info["other"].extend(
    [
        "ai_registered_until",
        "ai_import_country",
        "ai_sales_method",
        "ai_credit",
        "ai_deposit",
        "ai_installment_no",
        "ai_installment_amount",
        "ai_interest_free_credit",
        "ai_leasing",
        "ai_cash_payment",
        "ai_range_on_full_battery_km",
    ]
)

In [ ]:
df[features_info["numerical"]] = pd.to_numeric(df[features_info["numerical"]], downcast="unsigned")
df = downcast_nonnumerical_dtypes(df, features_info["binary"], features_info["ordinal"], features_info["nominal"])

## Testing

In [31]:
class TestNotebook:
    # @pytest.fixture
    def df(self):
        df = get_dataset_from_db()
        return df

    # @pytest.fixture
    def features_info(self):
        features_info = get_metadata()
        return features_info

    def run_cf_test(self, func_name_ua):
        df, features_info = self.df(), self.features_info()
        func_name_nb = f"{func_name_ua}{nb_utils.NB_SUFFIX}"

        uacleaner_obj = UACleaner()

        # Get function from class object from .py file
        func_ua = getattr(uacleaner_obj, func_name_ua)
        # Get function defined in this notebook
        func_nb = globals()[func_name_nb]

        df_ua, features_info_ua = func_ua(df=df, features_info=features_info)
        df_nb, features_info_nb = func_nb(df=df, features_info=features_info)
        
        pd.testing.assert_frame_equal(df_ua, df_nb)
        assert features_info_ua == features_info_nb

    def test_c_general_informations(self):
        df, features_info = self.df(), self.features_info()
        
        uacleaner_obj = UACleaner()

        df_ua, features_info_ua = uacleaner_obj.initial_clean(df=df, features_info=features_info)
        df_nb, features_info_nb = initial_clean_nb(df=df, features_info=features_info)

        df_ua, features_info_ua = uacleaner_obj.c_general_informations(df=df_ua, features_info=features_info_ua)
        df_nb, features_info_nb = c_general_informations_nb(df=df_nb, features_info=features_info_nb)
        
        pd.testing.assert_frame_equal(df_ua, df_nb)
        assert features_info_ua == features_info_nb

    def start(self):
        # Get all methods in UACleaner class with prefix 'cf_'
        cf_methods = [method for method in dir(UACleaner) if method.startswith('cf_') and callable(getattr(UACleaner, method))]

        # Run tests for each cf_method
        for cf_method in cf_methods:
            logging.info(cf_method)
            self.run_cf_test(cf_method)

        self.test_c_general_informations()

In [ ]:
test_obj = TestNotebook()
test_obj.start()

In [ ]:
uacleaner_obj = UACleaner()

df_ua = uacleaner_obj.clean(df=get_dataset_from_db())
features_info_ua = uacleaner_obj.features_info

pd.testing.assert_frame_equal(df, df_ua)
assert features_info == features_info_ua